In [1]:
import pandas as pd 
import numpy as np 
import plotly.express as px
from pathlib import Path
import plotly.graph_objects as go
from plotly.colors import sample_colorscale

In [2]:
glucose_df = pd.read_csv(Path('data') / 'Dexcom_004.csv', parse_dates=["Timestamp (YYYY-MM-DDThh:mm:ss)"])
glucose_df = glucose_df.dropna(subset=['Timestamp (YYYY-MM-DDThh:mm:ss)', 'Glucose Value (mg/dL)'])
glucose_df.head()

,Index,Timestamp (YYYY-MM-DDThh:mm:ss),Event Type,Event Subtype,Patient Info,Device Info,Source Device ID,Glucose Value (mg/dL),Insulin Value (u),Carb Value (grams),Duration (hh:mm:ss),Glucose Rate of Change (mg/dL/min),Transmitter Time (Long Integer)
11,12,2020-02-27 10:51:20,EGV,NaN,NaN,NaN,Android G6,124.0,NaN,NaN,NaN,NaN,8100.0
12,13,2020-02-27 10:56:20,EGV,NaN,NaN,NaN,Android G6,118.0,NaN,NaN,NaN,NaN,8400.0
13,14,2020-02-27 11:01:20,EGV,NaN,NaN,NaN,Android G6,113.0,NaN,NaN,NaN,NaN,8700.0
14,15,2020-02-27 11:06:20,EGV,NaN,NaN,NaN,Android G6,110.0,NaN,NaN,NaN,NaN,9000.0
15,16,2020-02-27 11:11:20,EGV,NaN,NaN,NaN,Android G6,107.0,NaN,NaN,NaN,NaN,9300.0


In [3]:
food_df = pd.read_csv(Path('data') / 'Food_Log_004.csv', parse_dates=["time_begin"])
food_df = food_df.dropna(subset=['time_begin', 'total_carb'])
food_df.head()

,date,time,time_begin,time_end,logged_food,amount,unit,searched_food,calorie,total_carb,dietary_fiber,sugar,protein,total_fat
0,2020-02-27,13:30:00,2020-02-27 13:30:00,NaN,Grapes,10.0,NaN,Grapes,34.0,8.9,0.4,7.6,0.4,0.1
1,2020-02-27,16:00:00,2020-02-27 16:00:00,NaN,Turkey Wings,2.0,NaN,Turkey Wings,852.0,0.0,0.0,0.0,102.0,46.0
2,2020-02-27,18:00:00,2020-02-27 18:00:00,NaN,Peach,1.0,NaN,Peach,68.0,17.0,2.6,15.0,1.6,0.4
3,2020-02-28,08:00:00,2020-02-28 08:00:00,NaN,Milk,4.0,fluid ounce,(Natrel) Lactose Free 2% Partly Skimmed Milk,60.0,4.5,0.0,4.0,6.0,2.5
4,2020-02-28,08:00:00,2020-02-28 08:00:00,NaN,Frosted Flakes,1.5,cup,(Kellogg's) Frosted Flakes,220.0,52.0,1.0,20.0,2.0,0.0


In [19]:
glucose_df['Timestamp (YYYY-MM-DDThh:mm:ss)']

11     2020-02-27 10:51:20
12     2020-02-27 10:56:20
13     2020-02-27 11:01:20
14     2020-02-27 11:06:20
15     2020-02-27 11:11:20
               ...        
2170   2020-03-06 06:31:05
2171   2020-03-06 06:36:05
2172   2020-03-06 06:41:06
2173   2020-03-06 06:46:05
2174   2020-03-06 06:51:06
Name: Timestamp (YYYY-MM-DDThh:mm:ss), Length: 2164, dtype: datetime64[ns]

In [34]:
food_df = food_df.drop_duplicates(subset=["time_begin"])

In [35]:
fig = px.line(glucose_df, x="Timestamp (YYYY-MM-DDThh:mm:ss)", y="Glucose Value (mg/dL)", title="Patient 004: Glucose Over Time")
fig.show()

In [37]:
# Choose a colorscale for carbs (e.g., Viridis, Reds, etc.)
# Normalize the total_carb values to map them to the color scale
max_carb = food_df["total_carb"].max()
min_carb = food_df["total_carb"].min()

# Map total_carb to a color scale (e.g., Reds)
colors = sample_colorscale(
    "Reds", 
    [(carb - min_carb) / (max_carb - min_carb) for carb in food_df["total_carb"]]
)

# Sort food_df by time to sync color order
food_df = food_df.sort_values("time_begin").reset_index(drop=True)

# Convert px figure to go.Figure to customize
fig = go.Figure(fig)

# Ensure time_begin is datetime and total_carb is numeric
food_df["time_begin"] = pd.to_datetime(food_df["time_begin"], errors="coerce")
food_df["total_carb"] = pd.to_numeric(food_df["total_carb"], errors="coerce")
food_df = food_df.dropna(subset=["time_begin", "total_carb"])

# Add vertical lines
for i, row in food_df.iterrows():
    carb = row["total_carb"]
    t = row["time_begin"].timestamp() * 1000

    fig.add_vline(
        x=t,  # Pass the numeric timestamp
        line=dict(color=colors[i], width=2, dash="dot"),
        annotation_text=f"{carb}g",
        annotation_position="top right",
        annotation_font_size=10,
        opacity=0.8
    )

# Update x-axis to interpret timestamps correctly
fig.update_layout(
    xaxis=dict(
        title="Time",
        type="date",  # Ensure Plotly interprets x-axis as datetime
    ),
    yaxis_title="Glucose (mg/dL)",
    legend_title="",
    title="Patient 004: Glucose with Meal Timing and Carbs"
)

fig.show()